
## Importing everything that is needed


In [1]:
import time
import yaml
import numpy as np
from abacusnbody.hod.abacus_hod import AbacusHOD
from astropy.table import Table, vstack
from cosmoprimo.utils import DistanceToRedshift
from pyrecon import utils
from cosmoprimo.fiducial import AbacusSummit
from scipy.interpolate import InterpolatedUnivariateSpline
import logging
import warnings
from tqdm import tqdm 
from astropy import units as u

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
logger = logging.getLogger("ds_abacus_lightcone")

## Define a function that return the ball sturcture for each halo catalogue. 
## HOD parameters are varied in log10, which need to be converted

In [2]:
def get_hod(p, param_mapping, param_tracer, Ball, nthread, seed):
    # read the parameters
    # print(p)
    for key in param_mapping.keys():
        mapping_idx = param_mapping[key]
        tracer_type = param_tracer[key]
        if key == "sigma" and tracer_type == "LRG":
            Ball.tracers[tracer_type][key] = 10 ** p[mapping_idx]
        elif key == "M_0" and tracer_type == "CSMF":
            Ball.tracers[tracer_type][key] = 10 ** p[mapping_idx] 
        elif key == "M_1" and tracer_type == "CSMF":
            Ball.tracers[tracer_type][key] = 10 ** p[mapping_idx] 
        elif key == "M_2" and tracer_type == "CSMF":
            Ball.tracers[tracer_type][key] = 10 ** p[mapping_idx]
        else:
            Ball.tracers[tracer_type][key] = p[mapping_idx]
    
    mock_dict = Ball.run_hod(tracers=Ball.tracers, want_rsd=Ball.want_rsd, want_nfw = False, NFW_draw=None, Nthread=nthread, reseed=seed, verbose=True)
    return mock_dict

## As the name it suggest it set ups the HOD code

In [3]:
def setup_hod(config):
    print(f"Processing {config['sim_params']['sim_name']}")
    sim_params = config["sim_params"]
    HOD_params = config["HOD_params"]
    fit_params = config["fit_params"]

    if(HOD_params['CSMF_params']['Mstar_low']<100):
        HOD_params['CSMF_params']['Mstar_low'] = 10**HOD_params['CSMF_params']['Mstar_low']
        HOD_params['CSMF_params']['Mstar_up'] = 10**HOD_params['CSMF_params']['Mstar_up']

    Balls = []
    for ez in zranges:
        sim_params["z_mock"] = ez
        Balls += [AbacusHOD(sim_params, HOD_params)]
        
    param_mapping = {}
    param_tracer = {}
    for key in fit_params.keys():
        mapping_idx = fit_params[key][0]
        tracer_type = fit_params[key][-1]
        param_mapping[key] = mapping_idx
        param_tracer[key] = tracer_type
        
    return Balls, param_mapping, param_tracer

## Dictionary to contral the code below 

In [4]:
args_dict = {
    "start_hod":0,
    "n_hod":1,
    "start_cosmo":0,
    "n_cosmo":1,
    "start_phase":0,
    "n_phase":1,
    "survey":"DESI",
    "tracer":"CSMF",
    "zmin":0.1,
    "zmax":0.4,
    "start_seed":0,
    "n_seed":1,
    "nthreads":5
}

## Load in the config file and define the redshifts that combined in the light cone

In [5]:
config = yaml.safe_load(open("data_CSMF/config_CSMF.yaml"))
zranges = [0.100]


## Downloading halo data using ``gdown``, which can be intalled with ``pip install gdown``


In [6]:
import gdown
gdown.download(id='1VCBXlkYHChn1HCwRPnq9iDSgzILfXfwL',output='data_CSMF/halos_data/AbacusSummit_base_c000_ph000/z0.100/particles_xcom_0_seed600_abacushod_oldfenv_MT_withranks_new.h5')
gdown.download(id='1xXlbXr8h8AiRFc2p5knYSYloJ9jfcAY9',output='data_CSMF/halos_data/AbacusSummit_base_c000_ph000/z0.100/halos_xcom_0_seed600_abacushod_oldfenv_MT_new.h5')

Downloading...
From (original): https://drive.google.com/uc?id=1VCBXlkYHChn1HCwRPnq9iDSgzILfXfwL
From (redirected): https://drive.google.com/uc?id=1VCBXlkYHChn1HCwRPnq9iDSgzILfXfwL&confirm=t&uuid=ce478b08-a653-4604-a55d-e2fb144f7ed9
To: /global/u2/p/pierre/abacusutils_modified/tests/data_CSMF/halos_data/AbacusSummit_base_c000_ph000/z0.100/particles_xcom_0_seed600_abacushod_oldfenv_MT_withranks_new.h5
100%|██████████| 260M/260M [00:01<00:00, 173MB/s]  
Downloading...
From (original): https://drive.google.com/uc?id=1xXlbXr8h8AiRFc2p5knYSYloJ9jfcAY9
From (redirected): https://drive.google.com/uc?id=1xXlbXr8h8AiRFc2p5knYSYloJ9jfcAY9&confirm=t&uuid=00ce9d81-134e-41b2-996c-c27bb162fa2c
To: /global/u2/p/pierre/abacusutils_modified/tests/data_CSMF/halos_data/AbacusSummit_base_c000_ph000/z0.100/halos_xcom_0_seed600_abacushod_oldfenv_MT_new.h5
100%|██████████| 108M/108M [00:00<00:00, 138MB/s]  


'data_CSMF/halos_data/AbacusSummit_base_c000_ph000/z0.100/halos_xcom_0_seed600_abacushod_oldfenv_MT_new.h5'

## Decide which cosmology, hod, seed and phase and prepare the HOD parameters

In [7]:
cosmo = 0
phase = 0
seed = 0
hod = 0

HOD_params = Table.read('data_CSMF/HOD_parameters.fits')
names = ['log_M_0', 'log_M_1', 'gamma_1', 'gamma_2', 'sigma_c','a_1', 'b_0', 'b_1', 's', 'Bcent', 'Bsat']

HOD_params_np = []
for name in names:
    HOD_params_np.append(np.array(HOD_params[name]))
HOD_params_np = np.array(HOD_params_np).T
print(HOD_params_np[hod])

[10.96635881 11.49130401 27.75835925  0.18179997  0.29946413  2.14532573
  0.41764339  0.46919005 -1.42349829 -0.49794641  0.53148565]


## Run the HOD code and write data to astropy Table

In [8]:
mock_cosmo = AbacusSummit(cosmo)
Balls, param_mapping, param_tracer = setup_hod(config)

data_positions_sky = []
for i, newBall in enumerate(Balls):

    hod_dict = get_hod(HOD_params_np[hod],param_mapping,param_tracer,newBall,args_dict['nthreads'],seed)
    data = hod_dict[args_dict['tracer']]

    dist, ra, dec = utils.cartesian_to_sky(np.c_[data['x'] + 990, data['y'] + 990, data['z'] + 990])
    d2z = DistanceToRedshift(mock_cosmo.comoving_radial_distance)
    redshift = d2z(dist)

    mask = (redshift >= args_dict['zmin']) & (redshift <= args_dict['zmax']) & (data['stellarmass']>0)
    central_id = np.zeros_like(data['stellarmass'])
    central_id[:data['Ncent']] = 1

    data_positions_sky.append(np.c_[ra[mask],dec[mask],dist[mask],redshift[mask],data['stellarmass'][mask],data['mass'][mask],central_id[mask]])
data_positions_sky = np.concatenate(data_positions_sky, axis=0)

HOD_table = Table()
HOD_table['ra'] = data_positions_sky[:,0].astype(np.float32)
HOD_table['dec'] = data_positions_sky[:,1].astype(np.float32)
HOD_table['comoving_distance_overh'] = data_positions_sky[:,2].astype(np.float32)*u.Mpc
HOD_table['redshift'] = data_positions_sky[:,3].astype(np.float32)
HOD_table['log10Mstar_overhsquare'] = np.log10(data_positions_sky[:,4]).astype(np.float64)*u.Msun
HOD_table['log10Mhalo_overh'] = np.log10(data_positions_sky[:,5]).astype(np.float64)*u.Msun
HOD_table['central flag'] = data_positions_sky[:,6].astype(np.int32)

HOD_table.write(f'data_CSMF/galaxy_catalogue_cos{cosmo:03}_ph{phase:03}_hod{hod:03}_seed{seed:03}.fits',overwrite=True)

Processing AbacusSummit_base_c000_ph000
generating centrals took  37.29109263420105
generating satellites took  44.424450635910034
CSMF number of galaxies  53496
satellite fraction  0.7775721549274712
organizing outputs took  5.719436407089233
generated CSMFs: 53496 satellite fraction  0.7775721549274712


In [9]:
HOD_table

ra,dec,comoving_distance_overh,redshift,log10Mstar_overhsquare,log10Mhalo_overh,central flag
,,Mpc,,solMass,solMass,
float32,float32,float32,float32,float64,float64,int32
89.80145,61.005356,313.05255,0.10718833,10.669699803548644,13.15851407014397,1
89.96813,15.081102,314.77386,0.10779407,10.661428342597803,12.567379512545882,1
89.60284,77.63843,297.2059,0.10162075,11.04491605968148,12.787388976424438,1
89.90948,79.4423,362.13913,0.12453798,10.457635730746278,12.501052347049345,1
89.011246,79.60034,358.28885,0.12317138,11.104613250739858,13.219736871286516,1
89.32601,75.91486,308.97787,0.10575519,10.88640251020487,13.195783032147952,1
89.75433,73.1615,343.47256,0.117921665,11.388144911160552,12.820884682162479,1
89.69066,62.50087,302.8254,0.10359327,11.11171486330106,13.362673792424015,1


## Download reference catalogue and compare it with the new generated output. Note that stellar masses are different for each run by construction.

In [10]:
from common import check_close

gdown.download(id='1JB5UvLM13wj6IdRvF37-Dh8kRjJWzWh0',output=f'data_CSMF/reference_galaxy_catalogue_cos{cosmo:03}_ph{phase:03}_hod{hod:03}_seed{seed:03}.fits')
HOD_table_ref = Table.read(f'data_CSMF/reference_galaxy_catalogue_cos{cosmo:03}_ph{phase:03}_hod{hod:03}_seed{seed:03}.fits')


for name in HOD_table_ref.colnames:
    if(check_close(HOD_table_ref[name],HOD_table[name])):
        print('\033[92m "'+name + r'" is close to the reference catalogue')
    else:
        print('\033[91m "'+name + r'" is different to the reference catalogue')

Downloading...
From: https://drive.google.com/uc?id=1JB5UvLM13wj6IdRvF37-Dh8kRjJWzWh0
To: /global/u2/p/pierre/abacusutils_modified/tests/data_CSMF/reference_galaxy_catalogue_cos000_ph000_hod000_seed000.fits
100%|██████████| 1.93M/1.93M [00:00<00:00, 16.1MB/s]


 "ra" is close to the reference catalogue
 "dec" is close to the reference catalogue
 "comoving_distance_overh" is close to the reference catalogue
 "redshift" is close to the reference catalogue
 "log10Mstar_overhsquare" is different to the reference catalogue
 "log10Mhalo_overh" is close to the reference catalogue
 "central flag" is close to the reference catalogue
